In [ ]:
# https://github.com/summer4an/mosaic_detector
import cv2
import numpy as np
from IPython.display import display
from PIL import Image
from time import time_ns

def d_gray(i):
    display(Image.fromarray(cv2.convertScaleAbs(i)))


def d_rgb(i):
    rgb = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
    display(Image.fromarray(rgb))


mosaic_conv = {}
point_conv = {}
for mosaic_size in range(10, 25):
    conv_size = 2 + mosaic_size + mosaic_size - 1 + 2
    img = np.ones((conv_size, conv_size, 1), np.uint8) * 255
    for i in range(2, conv_size, mosaic_size - 1):
        for j in range(2, conv_size, mosaic_size - 1):
            for k in range(0, conv_size):
                img[i, k] = 0
                img[k, j] = 0
    mosaic_conv.update({mosaic_size: img})

    size_factor = 3 * 2 + 1
    conv_size = mosaic_size * (size_factor - 1) + 1
    img = np.zeros((conv_size, conv_size, 1), np.uint8)
    for i in range(size_factor):
        for j in range(size_factor):
            img[i * mosaic_size, j * mosaic_size] = 1
    point_conv.update({mosaic_size: img.astype(np.float64)})

image = cv2.imread("E:/Dataset/amakano2/720p/e_yuu17N_1.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = 255 - cv2.Canny(gray, 10, 20)
gray = cv2.GaussianBlur(gray, (3, 3), 0)


for size, conv in mosaic_conv.items():
    start = time_ns()
    match_conv = cv2.matchTemplate(gray, conv, cv2.TM_CCOEFF_NORMED) * 255
    print((time_ns()-start)/1e9)
    if np.max(match_conv) >= 150:
        print(size)
        re, threshold = cv2.threshold(match_conv, 100, 255, cv2.THRESH_BINARY)
        start = time_ns()
        blur = cv2.filter2D(
            threshold, -1, point_conv[size - 1], borderType=cv2.BORDER_CONSTANT
        )
        print((time_ns()-start)/1e9)
        loc = np.where(match_conv + np.clip(blur, 0, 40) >= 80)
        for pt in zip(*loc[::-1]):
            cv2.rectangle(
                image, pt, (pt[0] + 2 * size, pt[1] + 2 * size), (0, 0, 0), -1
            )
        break

d_rgb(image)